### NOTE: This assignment was due 11/15 at 11:59pm. I would like to use one of my grace days (of the 6 I have) on this assignment to turn it in by 11:59pm on 11/16.  
# Homework 7
**name:** Mina Stojanovic  
**github id:** minastoj  
**USC student id:** 4968308304  

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
import numpy as np
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import hamming_loss
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import hamming

import warnings
warnings.filterwarnings('ignore')

### Question 1
#### Decision Trees as Interpretable Models
(a) Download the Anuran Calls (MFCCs) Data Set from: https://archive.ics.uci.edu/ml/datasets/Anuran+Calls+%28MFCCs%29. Choose 70% of the data randomly as the training set.  
(b) Each instance has three labels: Families, Genus, and Species. Each of the labels has multiple classes. We wish to solve a multi-class and multi-label problem. One of the most important approaches to multi-label classification is to train a classifier for each label (binary relevance). We first try this approach:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;i. Research exact match and hamming score/ loss methods for evaluating multi-label classification and use them in evaluating the classifiers in this problem.  
**Exact Match:** For each instance, if all labels are correctly predicted, the score is 1; otherwise, it is 0. The final score is the mean of these values across all instances.  
**Hamming Score:** It is computed as the ratio of the number of correct labels to the total number of labels (considering both positive and negative labels).  
**Hamming Loss:** It is computed as the average number of label mismatches per instance, divided by the total number of labels.

In [20]:
data = training_data = pd.read_csv('../data/Frogs_MFCCs.csv', delimiter=',')

input = data.drop(['Family', 'Genus', 'Species', 'RecordID'], axis=1)
output = data[['Family', 'Genus', 'Species']]

X_train, X_test, Y_train, Y_test = train_test_split(input, output, test_size=0.3, random_state=42)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;ii. Train a SVM for each of the labels, using Gaussian kernels and one versus all classifiers. Determine the weight of the SVM penalty and the width of the Gaussian Kernel using 10 fold cross validation. You are welcome to try to solve the problem with both standardized and raw attributes and report the results.  

In [21]:
# REFERENCE: Site #1, #2
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  
    'gamma': np.arange(1, 2.5, 0.1) # initially had similar values to C, but saw that all models had the same gamma value
}

family_svm = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv = 10)
genus_svm = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv = 10)
species_svm = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv = 10)

# fit each model on ITS specific output
family_svm.fit(X_train, Y_train['Family'])
genus_svm.fit(X_train, Y_train['Genus'])
species_svm.fit(X_train, Y_train['Species'])

# penalty 
family_best_C = family_svm.best_params_['C']
genus_best_C = genus_svm.best_params_['C']
species_best_C = species_svm.best_params_['C']

# gamma (width)
family_best_gamma = family_svm.best_params_['gamma']
genus_best_gamma = genus_svm.best_params_['gamma']
species_best_gamma = species_svm.best_params_['gamma']

results = [] # best parameter results for each model
results.append({
        'label': 'Family',
        'C': family_best_C,
        'gamma': family_best_gamma
    })
results.append({
        'label': 'Genus',
        'C': genus_best_C,
        'gamma': genus_best_gamma
    })
results.append({
        'label': 'Species',
        'C': species_best_C,
        'gamma': species_best_gamma
    })

results_df = pd.DataFrame(results)
display(results_df)

,label,C,gamma
0,Family,10,2.3
1,Genus,100,1.5
2,Species,10,1.8


In [22]:
family_Y_pred = family_svm.predict(X_test)
genus_Y_pred = genus_svm.predict(X_test)
species_Y_pred = species_svm.predict(X_test)

family_hamming_score = 1 - hamming_loss(Y_test['Family'], family_Y_pred)
genus_hamming_score = 1 - hamming_loss(Y_test['Genus'], genus_Y_pred)
species_hamming_score = 1 - hamming_loss(Y_test['Species'], species_Y_pred)

gaussian_overall_hamming_score = (family_hamming_score + genus_hamming_score + species_hamming_score) / 3

num_exact_matches = 0
for i in range(0, len(family_Y_pred)):
    if family_Y_pred[i] == Y_test['Family'].iloc[i] and genus_Y_pred[i] == Y_test['Genus'].iloc[i] and species_Y_pred[i] == Y_test['Species'].iloc[i]:
        num_exact_matches += 1

gaussian_overall_exact_match_score = num_exact_matches / len(family_Y_pred)

results = []
results.append({
    'Overall Hamming Score': gaussian_overall_hamming_score,
    'Overall Exact Match Score': gaussian_overall_exact_match_score
})

results_df = pd.DataFrame(results)
display(results_df)

,Overall Hamming Score,Overall Exact Match Score
0,0.991972,0.987494


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;iii. Repeat 1(b)ii with L1-penalized SVMs. Remember to standardize the attributes. Determine the weight of the SVM penalty using 10 fold cross validation.  

In [23]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

param_grid = {
    'C': [1, 25, 50, 75, 100] # increased from above C value range since many models had the same C value
}

# REFERENCE: Site #3
family_svm = GridSearchCV(svm.LinearSVC(penalty='l1', dual=False), param_grid, cv = 10)
genus_svm = GridSearchCV(svm.LinearSVC(penalty='l1', dual=False), param_grid, cv = 10)
species_svm = GridSearchCV(svm.LinearSVC(penalty='l1', dual=False), param_grid, cv = 10)

family_svm.fit(X_train_scaled, Y_train['Family'])
genus_svm.fit(X_train_scaled, Y_train['Genus'])
species_svm.fit(X_train_scaled, Y_train['Species'])

family_best_C = family_svm.best_params_['C']
genus_best_C = genus_svm.best_params_['C']
species_best_C = species_svm.best_params_['C']

results = []
results.append({
        'label': 'Family',
        'C': family_best_C
    })
results.append({
        'label': 'Genus',
        'C': genus_best_C
    })
results.append({
        'label': 'Species',
        'C': species_best_C
    })

results_df = pd.DataFrame(results)
display(results_df)

# METRICS
family_Y_pred = family_svm.predict(X_test_scaled)
genus_Y_pred = genus_svm.predict(X_test_scaled)
species_Y_pred = species_svm.predict(X_test_scaled)

family_hamming_score = 1 - hamming_loss(Y_test['Family'], family_Y_pred)
genus_hamming_score = 1 - hamming_loss(Y_test['Genus'], genus_Y_pred)
species_hamming_score = 1 - hamming_loss(Y_test['Species'], species_Y_pred)

l1_overall_hamming_score = (family_hamming_score + genus_hamming_score + species_hamming_score) / 3

num_exact_matches = 0
for i in range(0, len(family_Y_pred)):
    if family_Y_pred[i] == Y_test['Family'].iloc[i] and genus_Y_pred[i] == Y_test['Genus'].iloc[i] and species_Y_pred[i] == Y_test['Species'].iloc[i]:
        num_exact_matches += 1

l1_overall_exact_match_score = num_exact_matches / len(family_Y_pred)

results = []
results.append({
    'Overall Hamming Score': l1_overall_hamming_score,
    'Overall Exact Match Score': l1_overall_exact_match_score
})

results_df = pd.DataFrame(results)
display(results_df)

,label,C
0,Family,1
1,Genus,25
2,Species,50


,Overall Hamming Score,Overall Exact Match Score
0,0.943184,0.912923


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;iv. Repeat 1(b)iii by using SMOTE or any other method you know to remedy class imbalance. Report your conclusions about the classifiers you trained.

In [24]:
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100]
}

# apply smote
smote = SMOTE(random_state=42)
X_train_smote_family, Y_train_smote_family = smote.fit_resample(X_train_scaled, Y_train['Family'])
X_train_smote_genus, Y_train_smote_genus = smote.fit_resample(X_train_scaled, Y_train['Genus'])
X_train_smote_species, Y_train_smote_species = smote.fit_resample(X_train_scaled, Y_train['Species'])

family_svm = GridSearchCV(svm.LinearSVC(penalty='l1', dual=False), param_grid, cv=10)
genus_svm = GridSearchCV(svm.LinearSVC(penalty='l1', dual=False), param_grid, cv=10)
species_svm = GridSearchCV(svm.LinearSVC(penalty='l1', dual=False), param_grid, cv=10)

family_svm.fit(X_train_smote_family, Y_train_smote_family)
genus_svm.fit(X_train_smote_genus, Y_train_smote_genus)
species_svm.fit(X_train_smote_species, Y_train_smote_species)

family_best_C = family_svm.best_params_['C']
genus_best_C = genus_svm.best_params_['C']
species_best_C = species_svm.best_params_['C']

results = []
results.append({
    'label': 'Family',
    'C': family_best_C
})
results.append({
    'label': 'Genus',
    'C': genus_best_C
})
results.append({
    'label': 'Species',
    'C': species_best_C
})

results_df = pd.DataFrame(results)
display(results_df)

,label,C
0,Family,10
1,Genus,10
2,Species,10


In [30]:
# METRICS
family_Y_pred = family_svm.predict(X_test_scaled)
genus_Y_pred = genus_svm.predict(X_test_scaled)
species_Y_pred = species_svm.predict(X_test_scaled)

family_hamming_score = 1 - hamming_loss(Y_test['Family'], family_Y_pred)
genus_hamming_score = 1 - hamming_loss(Y_test['Genus'], genus_Y_pred)
species_hamming_score = 1 - hamming_loss(Y_test['Species'], species_Y_pred)

smote_l1_overall_hamming_score = (family_hamming_score + genus_hamming_score + species_hamming_score) / 3

num_exact_matches = 0
for i in range(len(family_Y_pred)):
    if (family_Y_pred[i] == Y_test['Family'].iloc[i] and genus_Y_pred[i] == Y_test['Genus'].iloc[i] and species_Y_pred[i] == Y_test['Species'].iloc[i]):
        num_exact_matches += 1

smote_l1_overall_exact_match_score = num_exact_matches / len(family_Y_pred)

results = []
results.append({
    'Overall Hamming Score': smote_l1_overall_hamming_score,
    'Overall Exact Match Score': smote_l1_overall_exact_match_score
})

results_df = pd.DataFrame(results)
display(results_df)

,Overall Hamming Score,Overall Exact Match Score
0,0.923113,0.853173


In [26]:
results = []
results.append({
    'Model': 'Gaussian SVM',
    'Overall Hamming Score': gaussian_overall_hamming_score,
    'Overall Exact Match Score': gaussian_overall_exact_match_score
})
results.append({
    'Model': 'L1-penalized SVM',
    'Overall Hamming Score': l1_overall_hamming_score,
    'Overall Exact Match Score': l1_overall_exact_match_score
})
results.append({
    'Model': 'SMOTE L1-penalized SVM',
    'Overall Hamming Score': smote_l1_overall_hamming_score,
    'Overall Exact Match Score': smote_l1_overall_exact_match_score
})

results_df = pd.DataFrame(results)
display(results_df)
print("As it is clear to see based off of the table, the SVM with the Gaussian kernel performed best, in terms of hamming score and exact match score.")
print("When we applied L1 penalization, the model performed worse than the Gaussian SVM model.")
print("Furthermore, when we applied SMOTE to the L1 penalized SVM, the model performed worse than the Gaussian and L1-penalized SVM models.")

,Model,Overall Hamming Score,Overall Exact Match Score
0,Gaussian SVM,0.991972,0.987494
1,L1-penalized SVM,0.943184,0.912923
2,SMOTE L1-penalized SVM,0.923113,0.853173


As it is clear to see based off of the table, the SVM with the Gaussian kernel performed best, in terms of hamming score and exact match score.
When we applied L1 penalization, the model performed worse than the Gaussian SVM model.
Furthermore, when we applied SMOTE to the L1 penalized SVM, the model performed worse than the Gaussian and L1-penalized SVM models.


### Question 2
#### K-Means Clustering on a Multi-Class and Multi-Label Data Set
**Monte-Carlo Simulation:** Perform the following procedures 50 times, and report the average and standard deviation of the 50 Hamming Distances that you calculate.  
(a) Use k-means clustering on the whole Anuran Calls (MFCCs) Data Set (do not split the data into train and test, as we are not performing supervised learning in this exercise). Choose k ∈ {1, 2, . . . , 50} automatically based on one of the methods provided in the slides (CH or Gap Statistics or scree plots or Silhouettes) or any other method you know.  

In [27]:
best_k = 0
best_k_score = 0
k_values = range(2, 51)

# REFERENCE: Site #4
for k in k_values:
    kmeans = KMeans(n_clusters=k).fit(input)
    score = silhouette_score(input, kmeans.labels_)
    if score > best_k_score:
        best_k = k
        best_k_score = score

print('Best K:', best_k)
print('Best K Score:', best_k_score)

Best K: 4
Best K Score: 0.3787509343305295


(b) In each cluster, determine which family is the majority by reading the true labels. Repeat for genus and species.  

In [28]:
kmeans = KMeans(n_clusters=best_k, random_state=42).fit(input)
labels = kmeans.labels_
majorities = {'Label': ['Family', 'Genus', 'Species']}

for k in range(best_k):
    cluster_indices = np.where(labels == k)[0] # get indices of samples in this cluster

    family_true_labels = data.iloc[cluster_indices]['Family']
    genus_true_labels = data.iloc[cluster_indices]['Genus']
    species_true_labels = data.iloc[cluster_indices]['Species']

    family_mode = family_true_labels.mode()[0]
    genus_mode = genus_true_labels.mode()[0]
    species_mode = species_true_labels.mode()[0]

    temp = [family_mode, genus_mode, species_mode]
    majorities[f'Cluster {k + 1}'] = temp

majorities_df = pd.DataFrame(majorities)
display(majorities_df)

,Label,Cluster 1,Cluster 2,Cluster 3,Cluster 4
0,Family,Leptodactylidae,Hylidae,Dendrobatidae,Hylidae
1,Genus,Adenomera,Hypsiboas,Ameerega,Hypsiboas
2,Species,AdenomeraHylaedactylus,HypsiboasCinerascens,Ameeregatrivittata,HypsiboasCordobae


(c) Now for each cluster you have a majority label triplet (family, genus, species). Calculate the average Hamming distance, Hamming score, and Hamming loss between the true labels and the labels assigned by clusters.

In [29]:
hamming_distances = []
hamming_scores = []
hamming_losses = []

for n in range(50):
    # part (a)
    best_k = 0
    best_k_score = 0
    k_values = range(2, 51)

    for k in k_values:
        kmeans = KMeans(n_clusters=k).fit(input)
        score = silhouette_score(input, kmeans.labels_)
        if score > best_k_score:
            best_k = k
            best_k_score = score

    # part (b)
    kmeans = KMeans(n_clusters=best_k, random_state=42).fit(input)
    labels = kmeans.labels_

    majorities = {'Label': ['Family', 'Genus', 'Species']}
    
    true_labels = []
    predicted_labels = []

    for k in range(best_k):
        cluster_indices = np.where(labels == k)[0]

        family_true_labels = data.iloc[cluster_indices]['Family']
        genus_true_labels = data.iloc[cluster_indices]['Genus']
        species_true_labels = data.iloc[cluster_indices]['Species']

        family_mode = family_true_labels.mode()[0]
        genus_mode = genus_true_labels.mode()[0]
        species_mode = species_mode = species_true_labels.mode()[0]

        majorities[f'Cluster {k + 1}'] = [family_mode, genus_mode, species_mode]

        for idx in cluster_indices:
            true_labels.append([data.iloc[idx]['Family'], data.iloc[idx]['Genus'], data.iloc[idx]['Species']])
            predicted_labels.append([family_mode, genus_mode, species_mode])

    # the new stuff
    hamm_dist = 0
    for i in range(len(true_labels)):
        if true_labels[i] != predicted_labels[i]:
            hamm_dist += 1

    hamm_loss = hamm_dist / len(true_labels)
    hamming_losses.append(hamm_loss)

    hamm_dist /= len(true_labels)
    hamm_dist *= len(true_labels[0])
    hamming_distances.append(hamm_dist)
    
    hamm_score = 1 - hamm_loss
    hamming_scores.append(hamm_score)

# METRICS
average_hamming_distance = np.mean(hamming_distances)
std_dev_hamming_distance = np.std(hamming_distances)
average_hamming_score = np.mean(hamming_scores)
average_hamming_loss = np.mean(hamming_losses)

results = [{
    'Average Hamming Distance': average_hamming_distance,
    'Standard Deviation of Hamming Distance': std_dev_hamming_distance,
    'Average Hamming Score': average_hamming_score,
    'Average Hamming Loss': average_hamming_loss
}]

results_df = pd.DataFrame(results)
display(results_df)


,Average Hamming Distance,Standard Deviation of Hamming Distance,Average Hamming Score,Average Hamming Loss
0,0.733426,0.0,0.755525,0.244475


### Question 3
#### ISLR 12.6.2
2. Suppose that we have four observations, for which we compute a dissimilarity matrix, given by  
[ 0, 0.3, 0.4, 0.7 ]  
[ 0.3, 0, 0.5, 0.8 ]  
[0.4, 0.5, 0, 0.45]  
[0.7, 0.8, 0.45, 0]  
For instance, the dissimilarity between the first and second observations is 0.3, and the dissimilarity between the second and fourth observations is 0.8.  
(a) On the basis of this dissimilarity matrix, sketch the dendrogram that results from hierarchically clustering these four observations using complete linkage. Be sure to indicate on the plot the height at which each fusion occurs, as well as the observations corresponding to each leaf in the dendrogram.  

![a.png](images/a.jpg)  

(b) Repeat (a), this time using single linkage clustering.  

![b.png](images/b.jpg)  

(c) Suppose that we cut the dendrogram obtained in (a) such that two clusters result. Which observations are in each cluster?

![c.png](images/c.jpg)  

(d) Suppose that we cut the dendrogram obtained in (b) such that two clusters result. Which observations are in each cluster?

![d.png](images/d.jpg)  

(e) It is mentioned in the chapter that at each fusion in the dendrogram, the position of the two clusters being fused can be swapped without changing the meaning of the dendrogram. Draw a dendrogram that is equivalent to the dendrogram in (a), for which two or more of the leaves are repositioned, but for which the meaning of the dendrogram is the same.

![e.png](images/e.jpg)  

## References and Citations  
[#1 - SVM Classifier Python Tutorial](https://www.datacamp.com/tutorial/svm-classification-scikit-learn-python)  
[#2 - SVM with GridSearchCV Tutorial](https://www.geeksforgeeks.org/svm-hyperparameter-tuning-using-gridsearchcv-ml/)  
[#3 - LinearSVC Documentation](https://scikit-learn.org/dev/modules/generated/sklearn.svm.LinearSVC.html)  
[#4 - KMeans Documentation](https://scikit-learn.org/1.5/modules/generated/sklearn.cluster.KMeans.html)  